<a href="https://colab.research.google.com/github/joshuadollison/smallbizpulse/blob/jd-model/notebooks/model_exploration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [2]:
# ============================================================
# SETUP: Mount Drive, Install Dependencies, Configure Styling
# ============================================================

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Install VADER for sentiment analysis
!pip install vaderSentiment -q

import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import seaborn as sns
from collections import Counter
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

# ── Consistent Plot Styling ──────────────────────────────────
plt.rcParams.update({
    'figure.figsize': (12, 6),
    'figure.dpi': 120,
    'font.family': 'sans-serif',
    'font.size': 11,
    'axes.titlesize': 14,
    'axes.titleweight': 'bold',
    'axes.labelsize': 12,
    'axes.spines.top': False,
    'axes.spines.right': False,
    'axes.grid': True,
    'grid.alpha': 0.3,
    'grid.linestyle': '--',
})

# SmallBizPulse color palette
COLORS = {
    'primary': '#2563EB',
    'secondary': '#F59E0B',
    'open': '#10B981',
    'closed': '#EF4444',
    'accent1': '#8B5CF6',
    'accent2': '#EC4899',
    'neutral': '#6B7280',
    'bg': '#F9FAFB',
}
PALETTE_OC = [COLORS['open'], COLORS['closed']]

print("Setup complete — libraries loaded, styling configured.")

Mounted at /content/drive
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 kB 9.5 MB/s eta 0:00:00
Setup complete — libraries loaded, styling configured.


In [3]:
# ============================================================
# DATA LOADING
# ============================================================
# >>> UPDATE THIS PATH to match your Google Drive folder <<<
DATA_PATH = '/content/drive/MyDrive/Colab Notebooks/CIS509/yelp_dataset_new/'

def load_json(filename):
    filepath = DATA_PATH + filename
    with open(filepath, 'r') as f:
        first_char = f.read(1)
        f.seek(0)
        if first_char == '[':
            return pd.DataFrame(json.load(f))
        else:
            return pd.read_json(f, lines=True)

print("Loading datasets...")
business_df = load_json('yelp_academic_dataset_business.json')
print(f"  Business: {len(business_df):,} records")

review_df = load_json('yelp_academic_dataset_review.json')
print(f"  Review:   {len(review_df):,} records")

tip_df = load_json('yelp_academic_dataset_tip.json')
print(f"  Tip:      {len(tip_df):,} records")

checkin_df = load_json('yelp_academic_dataset_checkin.json')
print(f"  Checkin:  {len(checkin_df):,} records")

user_df = load_json('yelp_academic_dataset_user.json')
print(f"  User:     {len(user_df):,} records")

print("\nAll datasets loaded successfully.")

Loading datasets...
  Business: 9,973 records
  Review:   100,000 records
  Tip:      264,693 records
  Checkin:  9,337 records
  User:     79,345 records

All datasets loaded successfully.


In [4]:
# ============================================================
# DATA SOURCES & FILTERING CRITERIA
# ============================================================

# Step 1: Filter for restaurants
# A business is classified as a "restaurant" if its Yelp categories
# contain the word "Restaurants" (case-insensitive).
restaurant_df = business_df[
    business_df['categories'].str.contains('Restaurants', case=False, na=False)
].copy()

# Step 2: Get restaurant business IDs
restaurant_ids = set(restaurant_df['business_id'])

# Step 3: Filter reviews to restaurant-only
rest_review_df = review_df[review_df['business_id'].isin(restaurant_ids)].copy()
rest_review_df['date'] = pd.to_datetime(rest_review_df['date'])
rest_review_df['year'] = rest_review_df['date'].dt.year

# Step 4: Filter tips to restaurant-only
rest_tip_df = tip_df[tip_df['business_id'].isin(restaurant_ids)].copy()

# Step 5: Filter checkins to restaurant-only
rest_checkin_df = checkin_df[checkin_df['business_id'].isin(restaurant_ids)].copy()

# Step 6: Merge business status onto reviews
status_map = restaurant_df.set_index('business_id')['is_open'].to_dict()
rest_review_df['is_open'] = rest_review_df['business_id'].map(status_map)
rest_review_df['status'] = rest_review_df['is_open'].map({1: 'Open', 0: 'Closed'})

# ── Print Summary ────────────────────────────────────────────
print("=" * 65)
print("DATA SOURCES & FILTERING SUMMARY")
print("=" * 65)

print("\nPRIMARY DATA SOURCE: Yelp Academic Dataset")
print("-" * 45)

print("\nFull Dataset:")
print(f"  Businesses:  {len(business_df):>8,}")
print(f"  Reviews:     {len(review_df):>8,}")
print(f"  Tips:        {len(tip_df):>8,}")
print(f"  Check-ins:   {len(checkin_df):>8,}")
print(f"  Users:       {len(user_df):>8,}")

print("\nFiltered to Restaurants (categories contain 'Restaurants'):")
print(f"  Restaurants:        {len(restaurant_df):>8,}")
print(f"  Restaurant Reviews: {len(rest_review_df):>8,}")
print(f"  Restaurant Tips:    {len(rest_tip_df):>8,}")
print(f"  Restaurant Checkins:{len(rest_checkin_df):>8,}")

n_open = (restaurant_df['is_open'] == 1).sum()
n_closed = (restaurant_df['is_open'] == 0).sum()
print("\nRestaurant Status:")
print(f"  Open:   {n_open:>5,}  ({n_open / len(restaurant_df) * 100:.1f}%)")
print(f"  Closed: {n_closed:>5,}  ({n_closed / len(restaurant_df) * 100:.1f}%)")

print(f"\nDate Range: {rest_review_df['date'].min().strftime('%Y-%m-%d')} to "
      f"{rest_review_df['date'].max().strftime('%Y-%m-%d')}")

print("\nFILTERING CRITERIA APPLIED:")
print("  1. Category filter: categories.str.contains('Restaurants')")
print("  2. Reviews, tips, and check-ins filtered by restaurant business_id")
print("  3. No minimum review count threshold (preserving data-sparse")
print("     businesses is important for studying closure patterns)")
print("  4. No date range restriction (full temporal span needed for time-series)")

DATA SOURCES & FILTERING SUMMARY

PRIMARY DATA SOURCE: Yelp Academic Dataset
---------------------------------------------

Full Dataset:
  Businesses:     9,973
  Reviews:      100,000
  Tips:         264,693
  Check-ins:      9,337
  Users:         79,345

Filtered to Restaurants (categories contain 'Restaurants'):
  Restaurants:           4,132
  Restaurant Reviews:   72,124
  Restaurant Tips:      20,394
  Restaurant Checkins:   4,085

Restaurant Status:
  Open:   2,575  (62.3%)
  Closed: 1,557  (37.7%)

Date Range: 2005-03-01 to 2018-10-04

FILTERING CRITERIA APPLIED:
  1. Category filter: categories.str.contains('Restaurants')
  2. Reviews, tips, and check-ins filtered by restaurant business_id
  3. No minimum review count threshold (preserving data-sparse
     businesses is important for studying closure patterns)
  4. No date range restriction (full temporal span needed for time-series)


# Get some counts

- wanted to see counts per month to get a sense of the types of model and windows we would want for regression/time-series

In [5]:
import pandas as pd

# Safety - ensure datetime (EDA notebook already does this, but this won't hurt)
rest_review_df['date'] = pd.to_datetime(rest_review_df['date'], errors='coerce')

# 1) Overall monthly review counts
monthly_counts = (
    rest_review_df
      .dropna(subset=['date'])
      .groupby(rest_review_df['date'].dt.to_period('M'))
      .size()
      .rename('review_count')
      .reset_index(name='review_count')
      .rename(columns={'date': 'month'})
)

# Convert Period to timestamp for easy plotting/merging (month start)
monthly_counts['month'] = monthly_counts['month'].dt.to_timestamp()

print(monthly_counts.head(12))
print('\nRows:', len(monthly_counts))
print('Date range:', monthly_counts['month'].min(), 'to', monthly_counts['month'].max())

# 2) Monthly counts split by business status (Open vs Closed) - if you created 'status' in EDA
if 'status' in rest_review_df.columns:
    monthly_counts_by_status = (
        rest_review_df
          .dropna(subset=['date'])
          .groupby([rest_review_df['date'].dt.to_period('M'), 'status'])
          .size()
          .rename('review_count')
          .reset_index()
          .rename(columns={'date': 'month'})
    )
    monthly_counts_by_status['month'] = monthly_counts_by_status['month'].dt.to_timestamp()
    print('\nBy status:')
    print(monthly_counts_by_status.head(12))

# 3) Monthly counts per business_id (useful for later time-series modeling)
monthly_counts_by_business = (
    rest_review_df
      .dropna(subset=['date'])
      .groupby(['business_id', rest_review_df['date'].dt.to_period('M')])
      .size()
      .rename('review_count')
      .reset_index()
      .rename(columns={'date': 'month'})
)
monthly_counts_by_business['month'] = monthly_counts_by_business['month'].dt.to_timestamp()

print('\nPer business:')
print(monthly_counts_by_business.head(12))

        month  review_count
0  2005-03-01             4
1  2005-04-01             3
2  2005-05-01             4
3  2005-06-01             1
4  2005-07-01            14
5  2005-08-01             1
6  2005-09-01             7
7  2005-10-01             2
8  2005-11-01             7
9  2005-12-01             4
10 2006-01-01            13
11 2006-02-01             5

Rows: 156
Date range: 2005-03-01 00:00:00 to 2018-10-01 00:00:00

By status:
        month  status  review_count
0  2005-03-01  Closed             1
1  2005-03-01    Open             3
2  2005-04-01  Closed             2
3  2005-04-01    Open             1
4  2005-05-01  Closed             2
5  2005-05-01    Open             2
6  2005-06-01    Open             1
7  2005-07-01  Closed             7
8  2005-07-01    Open             7
9  2005-08-01    Open             1
10 2005-09-01  Closed             3
11 2005-09-01    Open             4

Per business:
               business_id      month  review_count
0   --ZVrH2X2QXBFdCilbi

In [6]:
# Pivot for quick plot-ready table (month rows, status columns)
if 'status' in rest_review_df.columns:
    pivot = monthly_counts_by_status.pivot(index='month', columns='status', values='review_count').fillna(0).astype(int)
    print(pivot.tail(12))


status      Closed  Open
month                   
2017-11-01     129   529
2017-12-01     181   904
2018-01-01     141   652
2018-02-01     202   888
2018-03-01     220  1035
2018-04-01     202  1012
2018-05-01     177   924
2018-06-01     144   844
2018-07-01     182   967
2018-08-01     137   799
2018-09-01     105   749
2018-10-01       8    38


In [7]:
import pandas as pd

# Ensure datetime
rest_review_df['date'] = pd.to_datetime(rest_review_df['date'], errors='coerce')
df = rest_review_df.dropna(subset=['date']).copy()

# Ensure status exists (Open/Closed)
if 'status' not in df.columns:
    if 'is_open' in df.columns:
        df['status'] = df['is_open'].map({1: 'Open', 0: 'Closed'})
    else:
        raise ValueError("Need either 'status' or 'is_open' in rest_review_df.")

# Build year-month grain counts (so the averaging is fair across years)
df['year'] = df['date'].dt.year
df['month_num'] = df['date'].dt.month
df['month_name'] = df['date'].dt.strftime('%b')  # Jan, Feb, ...

monthly_counts = (
    df.groupby(['status', 'year', 'month_num', 'month_name'])
      .size()
      .reset_index(name='review_count')
)

# Average by calendar month across years
avg_by_month = (
    monthly_counts.groupby(['status', 'month_num', 'month_name'])['review_count']
      .mean()
      .reset_index(name='avg_reviews_per_month')
      .sort_values(['month_num', 'status'])
)

# Nice pivot view (rows = month, cols = status)
avg_by_month_pivot = (
    avg_by_month.pivot(index=['month_num', 'month_name'], columns='status', values='avg_reviews_per_month')
      .reset_index()
      .sort_values('month_num')
)

print(avg_by_month_pivot)

status  month_num month_name      Closed        Open
0               1        Jan  133.230769  343.076923
1               2        Feb  159.300000  356.083333
2               3        Mar  148.333333  392.833333
3               4        Apr  121.538462  348.230769
4               5        May  131.000000  358.285714
5               6        Jun  118.307692  318.000000
6               7        Jul  140.615385  404.461538
7               8        Aug  142.692308  367.571429
8               9        Sep  104.923077  320.833333
9              10        Oct  135.692308  342.153846
10             11        Nov  101.833333  273.000000
11             12        Dec  119.000000  298.166667


# Build the Business-Month Feature Table

We start by creating a clean monthly view of the reviews dataset.  First, we convert each review timestamp into a month bucket (YYYY-MM) so we can measure activity and behavior at a consistent time grain.  

We then compute:
1. Total monthly review volume split by business status (Open vs Closed)
2. A business-month feature table that aggregates review behavior for each restaurant each month (review count, average star rating, rating mix, engagement signals, and basic text length statistics).  

This business-month table becomes the backbone for the modeling pipeline - we will later enrich it with neural-network sentiment scores and BERTopic topic proportions, then feed sequences of monthly features into a GRU/RNN to predict future sentiment direction and closure risk.

In [8]:
import pandas as pd

df = rest_review_df.copy()

# Ensure datetime
df['date'] = pd.to_datetime(df['date'], errors='coerce')
df = df.dropna(subset=['date'])

# Month bucket (month start timestamp)
df['month'] = df['date'].dt.to_period('M').dt.to_timestamp()

# 1) Monthly totals by status (Open vs Closed)
monthly_by_status = (
    df.groupby(['status', 'month'])
      .size()
      .reset_index(name='review_count')
      .sort_values(['month', 'status'])
)

print(monthly_by_status.head(24))

# 2) Business-month backbone table (this is what the RNN will consume)
biz_month = (
    df.groupby(['business_id', 'status', 'month'])
      .agg(
          review_count=('review_id', 'count'),
          avg_stars=('stars', 'mean'),
          pct_1star=('stars', lambda s: (s <= 1.0).mean()),
          pct_5star=('stars', lambda s: (s >= 5.0).mean()),
          avg_useful=('useful', 'mean'),
          avg_funny=('funny', 'mean'),
          avg_cool=('cool', 'mean'),
          avg_text_len=('text', lambda x: x.fillna('').str.len().mean()),
          avg_word_count=('text', lambda x: x.fillna('').str.split().str.len().mean()),
      )
      .reset_index()
      .sort_values(['business_id', 'month'])
)

print(biz_month.head(20))

# 3) Optional: filter to businesses with enough activity for monthly sequences
# Example rule: at least 12 total business-month rows in the dataset
eligible = (
    biz_month.groupby('business_id')['month']
             .nunique()
             .reset_index(name='n_months')
)
eligible_ids = eligible.loc[eligible['n_months'] >= 12, 'business_id']

biz_month_eligible = biz_month[biz_month['business_id'].isin(eligible_ids)].copy()
print("Eligible businesses:", biz_month_eligible['business_id'].nunique())
print("Eligible rows:", len(biz_month_eligible))

     status      month  review_count
0    Closed 2005-03-01             1
150    Open 2005-03-01             3
1    Closed 2005-04-01             2
151    Open 2005-04-01             1
2    Closed 2005-05-01             2
152    Open 2005-05-01             2
153    Open 2005-06-01             1
3    Closed 2005-07-01             7
154    Open 2005-07-01             7
155    Open 2005-08-01             1
4    Closed 2005-09-01             3
156    Open 2005-09-01             4
5    Closed 2005-10-01             1
157    Open 2005-10-01             1
6    Closed 2005-11-01             2
158    Open 2005-11-01             5
7    Closed 2005-12-01             2
159    Open 2005-12-01             2
8    Closed 2006-01-01             7
160    Open 2006-01-01             6
9    Closed 2006-02-01             2
161    Open 2006-02-01             3
10   Closed 2006-03-01             4
162    Open 2006-03-01             6
               business_id  status      month  review_count  avg_stars  \
0

# VADER comparison

## 1) Create a baseline sentiment score (VADER) per review

This gives you an immediate, cheap sentiment channel to compare against the NN later.

In [9]:
import numpy as np
import pandas as pd

from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

df = rest_review_df.copy()
df['date'] = pd.to_datetime(df['date'], errors='coerce')
df = df.dropna(subset=['date'])

analyzer = SentimentIntensityAnalyzer()

# VADER scores
vader = df['text'].fillna('').apply(analyzer.polarity_scores)
df['vader_neg'] = vader.apply(lambda d: d['neg'])
df['vader_neu'] = vader.apply(lambda d: d['neu'])
df['vader_pos'] = vader.apply(lambda d: d['pos'])
df['vader_compound'] = vader.apply(lambda d: d['compound'])

df[['review_id','stars','vader_compound']].head()

,review_id,stars,vader_compound
0,KU_O5udG6zpxOg-VcAEodg,3.0,0.8597
2,saUsX_uimxRlCVr67Z4Jig,3.0,0.9201
3,AqPFMleE6RsU23_auESxiA,5.0,0.9588
4,Sx8TMOWLNuJBWer-0pcmoA,4.0,0.9815
5,JrIxlS1TzJ-iCu79ul40cQ,1.0,0.7117


## 2) Aggregate VADER to business-month features

This becomes part of the sequence input.

In [10]:
df['month'] = df['date'].dt.to_period('M').dt.to_timestamp()

biz_month_vader = (
    df.groupby(['business_id', 'status', 'month'])
      .agg(
          review_count=('review_id', 'count'),
          avg_stars=('stars', 'mean'),
          vader_mean=('vader_compound', 'mean'),
          vader_std=('vader_compound', 'std'),
          neg_share=('vader_compound', lambda s: (s < -0.05).mean()),
          pos_share=('vader_compound', lambda s: (s >  0.05).mean()),
      )
      .reset_index()
      .sort_values(['business_id', 'month'])
)

biz_month_vader.head(20)

,business_id,status,month,review_count,avg_stars,vader_mean,vader_std,neg_share,pos_share
0,--ZVrH2X2QXBFdCilbirsw,Closed,2013-07-01,1,5.0,0.8856,NaN,0.0,1.0
1,--ZVrH2X2QXBFdCilbirsw,Closed,2014-03-01,1,5.0,0.7777,NaN,0.0,1.0
2,--ZVrH2X2QXBFdCilbirsw,Closed,2014-12-01,1,5.0,0.8646,NaN,0.0,1.0
3,--ZVrH2X2QXBFdCilbirsw,Closed,2015-02-01,1,3.0,0.8921,NaN,0.0,1.0
4,--ZVrH2X2QXBFdCilbirsw,Closed,2015-05-01,1,5.0,0.6468,NaN,0.0,1.0
5,--ZVrH2X2QXBFdCilbirsw,Closed,2016-02-01,2,5.0,0.8821,0.076085,0.0,1.0
6,--ZVrH2X2QXBFdCilbirsw,Closed,2016-03-01,1,5.0,0.9449,NaN,0.0,1.0
7,--ZVrH2X2QXBFdCilbirsw,Closed,2017-07-01,1,5.0,0.9794,NaN,0.0,1.0
8,--ZVrH2X2QXBFdCilbirsw,Closed,2018-02-01,1,5.0,0.0000,NaN,0.0,0.0
9,-1MhPXk1FglglUAmuPLIGg,Open,2009-03-01,1,3.0,0.8020,NaN,0.0,1.0


# Sentiment Analysis

In this section we fine-tune a transformer-based sentiment model on our own Yelp review data to produce a high-quality, domain-specific sentiment signal.  We create a supervised training set using clearly polarized reviews (1-star = negative, 5-star = positive), split it into train/validation sets, and fine-tune DistilBERT to classify review sentiment.  After training, we use the best checkpoint to score every review with a continuous probability of positive sentiment (0-1).  Finally, we aggregate these transformer sentiment scores to the business-month level (mean, variability, and positive/negative share), creating time-series features that will later feed our GRU model for forecasting sentiment trajectories and predicting closure risk.

In [11]:
!pip -q install -U transformers datasets accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 68.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 515.2/515.2 kB 43.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.6/47.6 MB 55.3 MB/s eta 0:00:00


In [12]:
# ============================
# Cell 0 - Setup + Imports
# ============================
import os
import math
import random
import numpy as np
import pandas as pd

import torch
from datasets import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    DataCollatorWithPadding,
    TrainingArguments,
    Trainer
)

# ============================
# Cell 1 - Reproducibility
# ============================
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)

print("CUDA available:", torch.cuda.is_available())

# ============================
# Cell 2 - Prep Data
# Assumes rest_review_df is already loaded like your EDA notebook.
# ============================
df = rest_review_df.copy()

df["date"] = pd.to_datetime(df["date"], errors="coerce")
df = df.dropna(subset=["date"]).copy()

df["text"] = df["text"].fillna("").astype(str)

# Fine-tune labels: 1-star = 0 (negative), 5-star = 1 (positive)
train_df = df[df["stars"].isin([1.0, 5.0])].copy()
train_df["label"] = (train_df["stars"] == 5.0).astype(int)


train_df = df[df["stars"].isin([1.0, 2.0, 4.0, 5.0])].copy()
train_df["label"] = train_df["stars"].isin([4.0, 5.0]).astype(int)

print("Fine-tune rows:", len(train_df))
print(train_df["label"].value_counts())

# ============================
# Cell 3 - Stratified Train/Val Split (no sklearn)
# ============================
y = train_df["label"].values

idx_pos = np.where(y == 1)[0]
idx_neg = np.where(y == 0)[0]

np.random.shuffle(idx_pos)
np.random.shuffle(idx_neg)

split_pos = int(0.8 * len(idx_pos))
split_neg = int(0.8 * len(idx_neg))

tr_idx = np.concatenate([idx_pos[:split_pos], idx_neg[:split_neg]])
va_idx = np.concatenate([idx_pos[split_pos:], idx_neg[split_neg:]])

np.random.shuffle(tr_idx)
np.random.shuffle(va_idx)

train_split = train_df.iloc[tr_idx].reset_index(drop=True)
val_split = train_df.iloc[va_idx].reset_index(drop=True)

print("Train split:", len(train_split), "Val split:", len(val_split))
print("Train label dist:\n", train_split["label"].value_counts(normalize=True))
print("Val label dist:\n", val_split["label"].value_counts(normalize=True))

# ============================
# Cell 4 - Tokenize + Build HF Datasets
# ============================
MODEL_NAME = "distilbert-base-uncased"
MAX_LEN = 256

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

def tokenize_batch(batch):
    return tokenizer(
        batch["text"],
        truncation=True,
        max_length=MAX_LEN
    )

train_ds = Dataset.from_pandas(train_split[["text", "label"]])
val_ds = Dataset.from_pandas(val_split[["text", "label"]])

train_ds = train_ds.map(tokenize_batch, batched=True, remove_columns=["text"])
val_ds = val_ds.map(tokenize_batch, batched=True, remove_columns=["text"])

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# ============================
# Cell 5 - Model + Metrics
# ============================
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=2)

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=1)

    tp = int(((preds == 1) & (labels == 1)).sum())
    tn = int(((preds == 0) & (labels == 0)).sum())
    fp = int(((preds == 1) & (labels == 0)).sum())
    fn = int(((preds == 0) & (labels == 1)).sum())

    acc = (tp + tn) / max(1, tp + tn + fp + fn)
    precision = tp / max(1, tp + fp)
    recall = tp / max(1, tp + fn)
    f1 = 2 * precision * recall / max(1e-12, (precision + recall))

    return {"accuracy": acc, "precision": precision, "recall": recall, "f1": f1}

# ============================
# Cell 6 - TrainingArguments (Transformers v5-safe) + Trainer + Train
# ============================
OUT_DIR = "../artifacts/transformer_sentiment_distilbert"

EPOCHS = 3
PER_DEVICE_TRAIN_BS = 16
PER_DEVICE_EVAL_BS = 32
GRAD_ACCUM = 2

# warmup_ratio is deprecated in v5.2 - use warmup_steps now.
steps_per_epoch = math.ceil(len(train_ds) / (PER_DEVICE_TRAIN_BS * GRAD_ACCUM))
total_steps = steps_per_epoch * EPOCHS
warmup_steps = int(0.06 * total_steps)

training_args = TrainingArguments(
    output_dir=OUT_DIR,
    seed=SEED,

    # Transformers v5 uses eval_strategy (NOT evaluation_strategy)
    eval_strategy="epoch",
    save_strategy="epoch",

    load_best_model_at_end=True,
    metric_for_best_model="f1",
    greater_is_better=True,

    num_train_epochs=EPOCHS,
    learning_rate=2e-5,
    per_device_train_batch_size=PER_DEVICE_TRAIN_BS,
    per_device_eval_batch_size=PER_DEVICE_EVAL_BS,
    gradient_accumulation_steps=GRAD_ACCUM,

    warmup_steps=warmup_steps,
    weight_decay=0.01,

    logging_steps=100,
    fp16=torch.cuda.is_available(),
    report_to="none"
)

# Transformers v5 Trainer does NOT accept tokenizer=.  Use processing_class=.
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    data_collator=data_collator,
    processing_class=tokenizer,
    compute_metrics=compute_metrics
)

trainer.train()
eval_metrics = trainer.evaluate()
print("Eval metrics:", eval_metrics)

trainer.save_model(OUT_DIR)
tokenizer.save_pretrained(OUT_DIR)
print("Saved to:", OUT_DIR)

# ============================
# Cell 7 - Temperature scaling (calibration) on validation set
# Output: best_T
# ============================
pred_out = trainer.predict(val_ds)
val_logits = torch.tensor(pred_out.predictions, dtype=torch.float32)
val_labels = torch.tensor(pred_out.label_ids, dtype=torch.long)

def nll_for_T(T: float) -> float:
    scaled = val_logits / T
    probs = torch.softmax(scaled, dim=1)
    p = probs[torch.arange(len(val_labels)), val_labels]
    return (-torch.log(p.clamp_min(1e-12))).mean().item()

Ts = np.linspace(0.5, 5.0, 46)  # 0.5, 0.6, ..., 5.0
losses = [nll_for_T(float(T)) for T in Ts]
best_T = float(Ts[int(np.argmin(losses))])

print("Best temperature:", best_T)
print("NLL @ best_T:", min(losses))

# ============================
# Cell 8 - Score ALL Reviews (fast batch logits, temperature-scaled)
# Output: df['tx_sent'] = calibrated P(positive), 0..1
# ============================
from transformers import DataCollatorWithPadding

ft_model = AutoModelForSequenceClassification.from_pretrained(OUT_DIR)
ft_tokenizer = AutoTokenizer.from_pretrained(OUT_DIR)

use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
ft_model.to(device)
ft_model.eval()

collator = DataCollatorWithPadding(tokenizer=ft_tokenizer, return_tensors="pt")

texts = df["text"].fillna("").astype(str).tolist()
BATCH_SIZE = 64

p_pos = []

with torch.no_grad():
    for i in range(0, len(texts), BATCH_SIZE):
        batch_texts = texts[i:i + BATCH_SIZE]

        enc = ft_tokenizer(
            batch_texts,
            truncation=True,
            max_length=MAX_LEN
        )

        features = [{k: enc[k][j] for k in enc.keys()} for j in range(len(batch_texts))]
        batch = collator(features)
        batch = {k: v.to(device) for k, v in batch.items()}

        logits = ft_model(**batch).logits
        probs = torch.softmax(logits / best_T, dim=1)[:, 1].detach().cpu().numpy()
        p_pos.extend(probs.tolist())

df["tx_sent"] = np.array(p_pos, dtype=float)

print(df[["review_id", "stars", "tx_sent"]].head(10))
print("tx_sent range:", df["tx_sent"].min(), "to", df["tx_sent"].max())

# ============================
# Cell 9 - Aggregate to Business-Month Features (feeds GRU time-series later)
# Output: biz_month_tx
# ============================
df["month"] = df["date"].dt.to_period("M").dt.to_timestamp()

"""
biz_month_tx = (
    df.groupby(["business_id", "status", "month"])
      .agg(
          review_count=("review_id", "count"),
          avg_stars=("stars", "mean"),
          tx_sent_mean=("tx_sent", "mean"),
          tx_sent_std=("tx_sent", "std"),
          tx_neg_share=("tx_sent", lambda s: (s < 0.30).mean()),
          tx_pos_share=("tx_sent", lambda s: (s > 0.70).mean()),
      )
      .reset_index()
      .sort_values(["business_id", "month"])
)
"""

MIN_N = 5

biz_month_tx["tx_pos_share"] = np.where(
    biz_month_tx["review_count"] >= MIN_N,
    biz_month_tx["tx_pos_share"],
    np.nan
)

biz_month_tx["tx_neg_share"] = np.where(
    biz_month_tx["review_count"] >= MIN_N,
    biz_month_tx["tx_neg_share"],
    np.nan
)

# std is NaN when review_count == 1 - make it numeric for modeling
biz_month_tx["tx_sent_std"] = biz_month_tx["tx_sent_std"].fillna(0.0)

print(biz_month_tx.head(20))
print("Rows:", len(biz_month_tx), "Businesses:", biz_month_tx["business_id"].nunique())

ValueError: pyarrow.lib.IpcReadOptions size changed, may indicate binary incompatibility. Expected 112 from C header, got 104 from PyObject

In [ ]:
biz_month_tx["tx_neg_share"] = biz_month_tx["tx_neg_share"].fillna(0.5)
biz_month_tx["tx_pos_share"] = biz_month_tx["tx_pos_share"].fillna(0.5)

In [13]:
import sys, pyarrow, datasets
print("python:", sys.version)
print("pyarrow:", pyarrow.__version__, pyarrow.__file__)
print("datasets:", datasets.__version__)

ValueError: pyarrow.lib.IpcReadOptions size changed, may indicate binary incompatibility. Expected 112 from C header, got 104 from PyObject

# GRU

In [ ]:
# ============================
# Cell 10 - Build GRU-ready sequences (business-month time series)
# Goal: create X (sequences) + y (closure label) for modeling.
# Output: X (N, SEQ_LEN, F), y (N,), meta_df (one row per sequence)
# ============================

# ---- knobs ----
SEQ_LEN = 12          # months per sequence (1 year)
MIN_MONTHS = SEQ_LEN  # require at least SEQ_LEN months of history
TARGET_HORIZON = 1    # predict next month (we’ll label sequences by business status)
FEATURE_COLS = [
    "review_count",
    "avg_stars",
    "tx_sent_mean",
    "tx_sent_std",
    "tx_neg_share",
    "tx_pos_share",
]

# ---- safety checks ----
missing = [c for c in FEATURE_COLS if c not in biz_month_tx.columns]
if missing:
    raise ValueError(f"Missing required feature columns in biz_month_tx: {missing}")

# Ensure sorted
biz_month_tx = biz_month_tx.sort_values(["business_id", "month"]).reset_index(drop=True)

# Make sure numeric dtypes (GRU hates objects)
for c in FEATURE_COLS:
    biz_month_tx[c] = pd.to_numeric(biz_month_tx[c], errors="coerce")

# Fill any remaining NaNs defensively (should be minimal given your prior fills)
biz_month_tx[FEATURE_COLS] = biz_month_tx[FEATURE_COLS].fillna(0.0)

# ---- per-business normalization (recommended for sequences) ----
# This avoids one mega-popular restaurant dominating scale.
# We z-score each feature within each business.
def zscore_group(g: pd.DataFrame) -> pd.DataFrame:
    x = g[FEATURE_COLS].astype(float)
    mu = x.mean(axis=0)
    sd = x.std(axis=0).replace(0.0, 1.0)
    g[[f"{c}_z" for c in FEATURE_COLS]] = (x - mu) / sd
    return g

biz_month_tx = biz_month_tx.groupby("business_id", group_keys=False).apply(zscore_group)

FEATURE_Z = [f"{c}_z" for c in FEATURE_COLS]

# ---- build sequences ----
X_list = []
y_list = []
meta = []

# Label rule for now:
# y = 1 if business is Closed, else 0 (Open).
# (We’ll refine later to "predict closure in next k months".)
status_map = {"Closed": 1, "Open": 0}

for bid, g in biz_month_tx.groupby("business_id"):
    g = g.sort_values("month").reset_index(drop=True)

    # Need known status
    if g["status"].iloc[0] not in status_map:
        continue

    y_business = status_map[g["status"].iloc[0]]

    if len(g) < MIN_MONTHS:
        continue

    # Sliding windows: every possible SEQ_LEN-month chunk
    for start in range(0, len(g) - SEQ_LEN - TARGET_HORIZON + 1):
        end = start + SEQ_LEN
        window = g.iloc[start:end]

        X_seq = window[FEATURE_Z].to_numpy(dtype=np.float32)

        X_list.append(X_seq)
        y_list.append(y_business)
        meta.append({
            "business_id": bid,
            "status": g["status"].iloc[0],
            "start_month": window["month"].iloc[0],
            "end_month": window["month"].iloc[-1],
            "seq_len": SEQ_LEN,
        })

X = np.stack(X_list, axis=0) if len(X_list) else np.empty((0, SEQ_LEN, len(FEATURE_Z)), dtype=np.float32)
y = np.array(y_list, dtype=np.int64)
meta_df = pd.DataFrame(meta)

print("X shape:", X.shape)  # (num_sequences, SEQ_LEN, num_features)
print("y shape:", y.shape)
print("Class balance (y):", pd.Series(y).value_counts())
print(meta_df.head(10))

# ============================
# Cell 11 - Train/Val split at BUSINESS level (no leakage)
# Output: X_train, y_train, X_val, y_val
# ============================
SEED = 42
rng = np.random.default_rng(SEED)

unique_biz = meta_df["business_id"].unique()
rng.shuffle(unique_biz)

split = int(0.8 * len(unique_biz))
train_biz = set(unique_biz[:split])
val_biz   = set(unique_biz[split:])

train_mask = meta_df["business_id"].isin(train_biz).values
val_mask   = meta_df["business_id"].isin(val_biz).values

X_train, y_train = X[train_mask], y[train_mask]
X_val, y_val     = X[val_mask], y[val_mask]

print("Train sequences:", X_train.shape[0], "Val sequences:", X_val.shape[0])
print("Train y balance:\n", pd.Series(y_train).value_counts(normalize=True))
print("Val y balance:\n", pd.Series(y_val).value_counts(normalize=True))

# ============================
# Cell 12 - GRU model (Keras) for closure classification
# Output: trained GRU model + history
# ============================
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

tf.random.set_seed(SEED)

N_FEATS = X_train.shape[2]

model_gru = keras.Sequential([
    layers.Input(shape=(SEQ_LEN, N_FEATS)),
    layers.GRU(64, return_sequences=False),
    layers.Dropout(0.2),
    layers.Dense(32, activation="relu"),
    layers.Dropout(0.2),
    layers.Dense(1, activation="sigmoid"),
])

model_gru.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-3),
    loss="binary_crossentropy",
    metrics=[
        keras.metrics.BinaryAccuracy(name="acc"),
        keras.metrics.Precision(name="precision"),
        keras.metrics.Recall(name="recall"),
        keras.metrics.AUC(name="auc"),
    ],
)

# handle imbalance via class weights (Closed is usually minority)
pos = float((y_train == 1).sum())
neg = float((y_train == 0).sum())
w0 = (pos + neg) / max(1.0, 2.0 * neg)
w1 = (pos + neg) / max(1.0, 2.0 * pos)
class_weight = {0: w0, 1: w1}

callbacks = [
    keras.callbacks.EarlyStopping(monitor="val_auc", mode="max", patience=3, restore_best_weights=True),
    keras.callbacks.ReduceLROnPlateau(monitor="val_auc", mode="max", factor=0.5, patience=2, min_lr=1e-5),
]

history = model_gru.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=20,
    batch_size=128,
    class_weight=class_weight,
    callbacks=callbacks,
    verbose=1
)

val_metrics = model_gru.evaluate(X_val, y_val, verbose=0)
print(dict(zip(model_gru.metrics_names, val_metrics)))

# ============================
# Cell 13 - Quick sanity check: predicted risk examples
# Output: top-risk businesses (by avg predicted probability over their sequences)
# ============================
val_probs = model_gru.predict(X_val, batch_size=256).reshape(-1)

val_meta = meta_df.loc[val_mask].copy()
val_meta["p_closed"] = val_probs

biz_risk = (
    val_meta.groupby(["business_id", "status"])["p_closed"]
      .mean()
      .reset_index()
      .sort_values("p_closed", ascending=False)
)

print(biz_risk.head(20))
print(biz_risk.tail(20))



In [ ]:
# ============================
# Cell 14 - Create closure_month proxy per business
# For Closed businesses, closure_month = last review month we observed.
# ============================
biz_last_month = (
    biz_month_tx.groupby(["business_id", "status"])["month"]
      .max()
      .reset_index()
      .rename(columns={"month": "last_review_month"})
)

biz_last_month["closure_month"] = pd.NaT
biz_last_month.loc[biz_last_month["status"] == "Closed", "closure_month"] = biz_last_month.loc[
    biz_last_month["status"] == "Closed", "last_review_month"
].values

print(biz_last_month.head(10))
print("Closed businesses:", (biz_last_month["status"] == "Closed").sum())
print("Open businesses:", (biz_last_month["status"] == "Open").sum())

# ============================
# Cell 15 - Rebuild sequences with a proper horizon label
# y = 1 if business closes within H months AFTER the window ends
# ============================
SEQ_LEN = 12
H = 6  # predict closure within next 6 months

FEATURE_COLS = [
    "review_count",
    "avg_stars",
    "tx_sent_mean",
    "tx_sent_std",
    "tx_neg_share",
    "tx_pos_share",
]

# Ensure sorted
biz_month_tx = biz_month_tx.sort_values(["business_id", "month"]).reset_index(drop=True)

# Merge closure_month onto each row
biz_month_tx2 = biz_month_tx.merge(
    biz_last_month[["business_id", "status", "closure_month"]],
    on=["business_id", "status"],
    how="left"
)

# Build z-scored features per business again (same as before)
def zscore_group(g: pd.DataFrame) -> pd.DataFrame:
    x = g[FEATURE_COLS].astype(float)
    mu = x.mean(axis=0)
    sd = x.std(axis=0).replace(0.0, 1.0)
    g[[f"{c}_z" for c in FEATURE_COLS]] = (x - mu) / sd
    return g

biz_month_tx2 = biz_month_tx2.groupby("business_id", group_keys=False).apply(zscore_group)
FEATURE_Z = [f"{c}_z" for c in FEATURE_COLS]

# Activity filter: require at least SOME months with reviews in the window
MIN_ACTIVE_MONTHS = 6  # out of 12
MIN_REVIEWS_IN_WINDOW = 10

X_list, y_list, meta = [], [], []

for bid, g in biz_month_tx2.groupby("business_id"):
    g = g.sort_values("month").reset_index(drop=True)
    status = g["status"].iloc[0]
    closure_month = g["closure_month"].iloc[0]

    if len(g) < SEQ_LEN:
        continue

    for start in range(0, len(g) - SEQ_LEN + 1):
        end = start + SEQ_LEN
        window = g.iloc[start:end].copy()
        window_end = window["month"].iloc[-1]

        # window activity filters
        active_months = int((window["review_count"] > 0).sum())
        total_reviews = float(window["review_count"].sum())
        if active_months < MIN_ACTIVE_MONTHS:
            continue
        if total_reviews < MIN_REVIEWS_IN_WINDOW:
            continue

        # horizon label
        if status == "Open" or pd.isna(closure_month):
            y_seq = 0
        else:
            # label 1 if closure happens within next H months after window_end
            # and window_end is before closure (avoid post-closure nonsense)
            y_seq = int((window_end < closure_month) and (window_end >= (closure_month - pd.DateOffset(months=H))))

        X_seq = window[FEATURE_Z].to_numpy(dtype=np.float32)

        X_list.append(X_seq)
        y_list.append(y_seq)
        meta.append({
            "business_id": bid,
            "status": status,
            "start_month": window["month"].iloc[0],
            "end_month": window_end,
            "closure_month": closure_month,
            "y": y_seq
        })

X2 = np.stack(X_list, axis=0) if len(X_list) else np.empty((0, SEQ_LEN, len(FEATURE_Z)), dtype=np.float32)
y2 = np.array(y_list, dtype=np.int64)
meta2 = pd.DataFrame(meta)

print("X2 shape:", X2.shape)
print("y2 balance:", pd.Series(y2).value_counts())
print(meta2.head(10))

# ============================
# Cell 16 - Business-level split again (no leakage)
# ============================
rng = np.random.default_rng(42)
unique_biz = meta2["business_id"].unique()
rng.shuffle(unique_biz)

split = int(0.8 * len(unique_biz))
train_biz = set(unique_biz[:split])
val_biz   = set(unique_biz[split:])

train_mask = meta2["business_id"].isin(train_biz).values
val_mask   = meta2["business_id"].isin(val_biz).values

X_train2, y_train2 = X2[train_mask], y2[train_mask]
X_val2, y_val2     = X2[val_mask], y2[val_mask]

print("Train sequences:", X_train2.shape[0], "Val sequences:", X_val2.shape[0])
print("Train y balance:\n", pd.Series(y_train2).value_counts(normalize=True))
print("Val y balance:\n", pd.Series(y_val2).value_counts(normalize=True))

In [ ]:
# ============================
# Cell 17 - GRU (imbalance-safe) + Train
# Uses weighted BCE with pos_weight = neg/pos
# ============================
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

tf.random.set_seed(42)

SEQ_LEN = X_train2.shape[1]
N_FEATS = X_train2.shape[2]

pos = float((y_train2 == 1).sum())
neg = float((y_train2 == 0).sum())
pos_weight = neg / max(1.0, pos)

print("Train pos:", int(pos), "Train neg:", int(neg), "pos_weight:", pos_weight)

def weighted_bce(y_true, y_pred):
    # y_true: (batch,)
    # y_pred: (batch,)
    y_true = tf.cast(y_true, tf.float32)
    eps = tf.keras.backend.epsilon()
    y_pred = tf.clip_by_value(y_pred, eps, 1.0 - eps)

    # BCE = -(y*log(p) + (1-y)*log(1-p))
    # Weight positives by pos_weight
    loss = -(pos_weight * y_true * tf.math.log(y_pred) + (1.0 - y_true) * tf.math.log(1.0 - y_pred))
    return tf.reduce_mean(loss)

model_gru = keras.Sequential([
    layers.Input(shape=(SEQ_LEN, N_FEATS)),
    layers.GRU(96, return_sequences=False),
    layers.Dropout(0.25),
    layers.Dense(48, activation="relu"),
    layers.Dropout(0.25),
    layers.Dense(1, activation="sigmoid"),
])

model_gru.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-3),
    loss=weighted_bce,
    metrics=[
        keras.metrics.AUC(name="auc"),
        keras.metrics.Precision(name="precision"),
        keras.metrics.Recall(name="recall"),
    ],
)

callbacks = [
    keras.callbacks.EarlyStopping(monitor="val_auc", mode="max", patience=4, restore_best_weights=True),
    keras.callbacks.ReduceLROnPlateau(monitor="val_auc", mode="max", factor=0.5, patience=2, min_lr=1e-5),
]

history = model_gru.fit(
    X_train2, y_train2,
    validation_data=(X_val2, y_val2),
    epochs=30,
    batch_size=256,
    callbacks=callbacks,
    verbose=1
)

val_metrics = model_gru.evaluate(X_val2, y_val2, verbose=0)
print(dict(zip(model_gru.metrics_names, val_metrics)))

# ============================
# Cell 18 - Threshold sweep (pick operating point)
# ============================
import numpy as np

probs = model_gru.predict(X_val2, batch_size=512).reshape(-1)
ytrue = y_val2.astype(int)

thresholds = np.linspace(0.05, 0.95, 19)

rows = []
for t in thresholds:
    pred = (probs >= t).astype(int)
    tp = int(((pred == 1) & (ytrue == 1)).sum())
    fp = int(((pred == 1) & (ytrue == 0)).sum())
    fn = int(((pred == 0) & (ytrue == 1)).sum())

    precision = tp / max(1, (tp + fp))
    recall    = tp / max(1, (tp + fn))
    f1        = (2 * precision * recall) / max(1e-12, (precision + recall))
    rows.append((t, tp, fp, fn, precision, recall, f1))

print("thr  tp  fp  fn  precision  recall  f1")
for r in rows:
    print(f"{r[0]:.2f}  {r[1]:4d} {r[2]:4d} {r[3]:4d}   {r[4]:.3f}     {r[5]:.3f}  {r[6]:.3f}")

In [ ]:
# ============================
# Cell 19 - Workload-based evaluation (Top-K% triage) - FIXED for your variables
# Run AFTER Cell 16 (val_mask exists) and AFTER Cell 17/18 (model_gru trained).
# Requires: model_gru, X_val2, y_val2, meta2, val_mask
# ============================

import numpy as np
import pandas as pd

# 1) Predict probabilities on validation sequences
probs = model_gru.predict(X_val2, batch_size=512, verbose=0).reshape(-1)
ytrue = y_val2.astype(int)

# 2) Align validation metadata 1:1 with X_val2
val_meta2 = meta2.loc[val_mask].copy().reset_index(drop=True)

if len(val_meta2) != len(probs):
    raise ValueError(f"Meta/Pred mismatch.  val_meta2={len(val_meta2)} probs={len(probs)}")

val_meta2["p_closed"] = probs
val_meta2["y_true"] = ytrue

total_pos = int((val_meta2["y_true"] == 1).sum())
n = len(val_meta2)

print("Val windows:", n)
print("Val positives:", total_pos, "(", total_pos / max(1, n), ")")

# 3) Top-K% triage: if we only investigate the riskiest K% of windows, what's precision/recall?
print("\nTop-K% triage (higher p_closed = higher risk):")
for pct in [0.5, 1, 2, 5, 10]:
    k = max(1, int(n * (pct / 100.0)))
    topk = val_meta2.sort_values("p_closed", ascending=False).head(k)

    tp = int((topk["y_true"] == 1).sum())
    precision = tp / max(1, k)
    recall = tp / max(1, total_pos)

    print(f"Top {pct:>4}% (k={k:>5}): precision={precision:.3f}  recall={recall:.3f}  tp={tp}")

# 4) Sanity check lists
cols = [c for c in ["business_id", "status", "start_month", "end_month", "closure_month", "y_true", "p_closed"] if c in val_meta2.columns]

print("\nTop 20 highest-risk windows (sanity check):")
print(val_meta2.sort_values("p_closed", ascending=False)[cols].head(20))

print("\nBottom 20 lowest-risk windows (sanity check):")
print(val_meta2.sort_values("p_closed", ascending=True)[cols].head(20))

In [ ]:
# ============================
# Cell 20 - BUSINESS-level triage on the validation set (one score per business)
# Run AFTER Cell 19 (val_meta2 exists with: business_id, end_month, p_closed, y_true, status)
# Output: biz_val_scores + Top-K business triage metrics
# ============================

import numpy as np
import pandas as pd

# Safety: make sure we have what we need
need_cols = ["business_id", "end_month", "p_closed", "y_true"]
missing = [c for c in need_cols if c not in val_meta2.columns]
if missing:
    raise ValueError(f"val_meta2 missing required columns: {missing}")

# Ensure datetime sorting
val_meta2 = val_meta2.copy()
val_meta2["end_month"] = pd.to_datetime(val_meta2["end_month"])

# One row per business.  We compute:
# - p_last: score on most recent window (this is "risk now")
# - p_max: worst-case window score (good for "ever looked bad")
# - p_mean: average window score (stability)
# - y_business: business is positive if ANY window is positive in val (since y_true is per-window)
biz_val_scores = (
    val_meta2.sort_values(["business_id", "end_month"])
      .groupby("business_id", as_index=False)
      .agg(
          status=("status", "first") if "status" in val_meta2.columns else ("business_id", "size"),
          end_month_last=("end_month", "max"),
          p_last=("p_closed", "last"),
          p_max=("p_closed", "max"),
          p_mean=("p_closed", "mean"),
          y_business=("y_true", "max"),
          n_windows=("p_closed", "size"),
      )
)

total_pos_biz = int((biz_val_scores["y_business"] == 1).sum())
n_biz = len(biz_val_scores)

print("Val businesses:", n_biz)
print("Val positive businesses (has >=1 positive window):", total_pos_biz, "(", total_pos_biz / max(1, n_biz), ")")

# Choose the score you want to operate on:
# - p_last is the right default for "risk now"
SCORE_COL = "p_last"

print(f"\nBusiness-level Top-K% triage using {SCORE_COL}:")
for pct in [0.5, 1, 2, 5, 10]:
    k = max(1, int(n_biz * (pct / 100.0)))
    topk = biz_val_scores.sort_values(SCORE_COL, ascending=False).head(k)

    tp = int((topk["y_business"] == 1).sum())
    precision = tp / max(1, k)
    recall = tp / max(1, total_pos_biz)

    print(f"Top {pct:>4}% (k={k:>5}): precision={precision:.3f}  recall={recall:.3f}  tp={tp}")

print("\nTop 25 highest-risk businesses (risk now):")
cols = [c for c in ["business_id", "status", "end_month_last", "p_last", "p_max", "p_mean", "y_business", "n_windows"] if c in biz_val_scores.columns]
print(biz_val_scores.sort_values(SCORE_COL, ascending=False)[cols].head(25))

print("\nBottom 25 lowest-risk businesses (risk now):")
print(biz_val_scores.sort_values(SCORE_COL, ascending=True)[cols].head(25))

In [ ]:
# ============================
# Cell 21 - Add "recent max" score (stability / recent-risk safety net)
# Goal: catch businesses that spiked recently even if p_last is low.
# Requires: val_meta2 (window-level) and biz_val_scores from Cell 20
# ============================

import numpy as np
import pandas as pd

# knobs
RECENT_WINDOWS = 6  # last 6 windows (roughly last ~6 months of "ending months")

# Ensure time sort
vm = val_meta2.copy()
vm["end_month"] = pd.to_datetime(vm["end_month"])

# For each business, take last RECENT_WINDOWS windows by end_month, compute max prob
recent_max = (
    vm.sort_values(["business_id", "end_month"])
      .groupby("business_id", as_index=False)
      .tail(RECENT_WINDOWS)
      .groupby("business_id", as_index=False)["p_closed"]
      .max()
      .rename(columns={"p_closed": "p_recent_max"})
)

biz_val_scores2 = biz_val_scores.merge(recent_max, on="business_id", how="left")
biz_val_scores2["p_recent_max"] = biz_val_scores2["p_recent_max"].fillna(biz_val_scores2["p_last"])

# Optional: composite score (weighted toward "now", but respects recent spikes)
# You can adjust weights.  This is a sane default.
biz_val_scores2["p_combo"] = 0.7 * biz_val_scores2["p_last"] + 0.3 * biz_val_scores2["p_recent_max"]

print("Added columns: p_recent_max, p_combo")
print(biz_val_scores2[["business_id","p_last","p_recent_max","p_combo","y_business"]].head(10))

def topk_report(df, score_col, pcts=(0.5,1,2,5,10)):
    total_pos = int((df["y_business"] == 1).sum())
    n = len(df)
    print(f"\nTop-K% triage using {score_col}: (n={n}, pos={total_pos}, pos_rate={total_pos/max(1,n):.3f})")
    for pct in pcts:
        k = max(1, int(n * (pct / 100.0)))
        topk = df.sort_values(score_col, ascending=False).head(k)
        tp = int((topk["y_business"] == 1).sum())
        precision = tp / max(1, k)
        recall = tp / max(1, total_pos)
        print(f"Top {pct:>4}% (k={k:>5}): precision={precision:.3f}  recall={recall:.3f}  tp={tp}")

topk_report(biz_val_scores2, "p_last")
topk_report(biz_val_scores2, "p_recent_max")
topk_report(biz_val_scores2, "p_combo")

print("\nTop 25 by p_combo:")
cols = [c for c in ["business_id","status","end_month_last","p_last","p_recent_max","p_combo","p_max","p_mean","y_business","n_windows"] if c in biz_val_scores2.columns]
print(biz_val_scores2.sort_values("p_combo", ascending=False)[cols].head(25))

print("\nBottom 25 by p_combo:")
print(biz_val_scores2.sort_values("p_combo", ascending=True)[cols].head(25))

In [ ]:
# ============================
# Cell 22 - Final triage table (business-level) + export
# Uses p_recent_max as the primary risk score.
# Requires: biz_val_scores2 (from Cell 21)
# ============================

import pandas as pd

triage = biz_val_scores2.copy()

# Primary score
triage["risk_score"] = triage["p_recent_max"]

# Helpful derived fields
triage["risk_bucket"] = pd.cut(
    triage["risk_score"],
    bins=[-1, 0.2, 0.4, 0.6, 0.8, 1.01],
    labels=["very_low", "low", "medium", "high", "very_high"]
)

# Sort for action
triage = triage.sort_values(["risk_score", "p_last"], ascending=False)

cols = [c for c in [
    "business_id",
    "status",
    "end_month_last",
    "risk_score",
    "p_recent_max",
    "p_last",
    "p_max",
    "p_mean",
    "n_windows",
    "risk_bucket",
    "y_business",
] if c in triage.columns]

triage_out = triage[cols].reset_index(drop=True)

print("Triage rows:", len(triage_out))
print(triage_out.head(30))

# Export (optional but recommended)
OUT_PATH = "../artifacts/gru_business_triage.csv"
triage_out.to_csv(OUT_PATH, index=False)
print("Saved:", OUT_PATH)

In [ ]:
# ============================
# Cell 23 - Workload cutlines (business triage) + export top lists
# Run right after Cell 22 (needs triage_out from Cell 22)
# ============================

import numpy as np

df = triage_out.copy()

score_col = "risk_score"   # this is p_recent_max
y_col = "y_business"

n = len(df)
pos = int((df[y_col] == 1).sum())
pos_rate = pos / max(1, n)

print(f"Businesses: {n}  Positives: {pos}  Pos rate: {pos_rate:.3f}")
print("\nTop-K% workload metrics (sorted by risk_score desc):")

df = df.sort_values(score_col, ascending=False).reset_index(drop=True)

for pct in [0.5, 1, 2, 5, 10, 15, 20]:
    k = max(1, int(round(n * (pct / 100.0))))
    topk = df.head(k)

    tp = int((topk[y_col] == 1).sum())
    precision = tp / max(1, k)
    recall = tp / max(1, pos)

    # threshold at this cut (lowest score inside topk)
    thr = float(topk[score_col].min())

    print(f"Top {pct:>4}% (k={k:>4})  thr>={thr:.4f}  precision={precision:.3f}  recall={recall:.3f}  tp={tp}")

# Export a couple ready-to-use lists
for pct in [5, 10]:
    k = max(1, int(round(n * (pct / 100.0))))
    out = df.head(k).copy()
    out_path = f"../artifacts/gru_business_triage_top{pct}pct.csv"
    out.to_csv(out_path, index=False)
    print("Saved:", out_path)

In [ ]:
# ============================
# ONE CELL - Score ALL businesses, aggregate to business-level triage, save CSVs
# Run AFTER your Cell 17 training (model_gru exists) AND after Cell 16 (X2, y2, meta2, train_mask/val_mask exist)
# Outputs:
#   ../artifacts/gru_business_triage_all.csv
#   ../artifacts/gru_business_triage_all_top5pct.csv
#   ../artifacts/gru_business_triage_all_top10pct.csv
# ============================

import os
import numpy as np
import pandas as pd

# ---- required objects ----
required = ["model_gru", "X2", "y2", "meta2"]
missing = [r for r in required if r not in globals()]
if missing:
    raise RuntimeError(f"Missing required objects in memory: {missing}.  "
                       f"Run your sequence build (Cell 15/16) and GRU training (Cell 17) first.")

# ---- ensure artifacts dir ----
OUT_DIR = "../artifacts"
os.makedirs(OUT_DIR, exist_ok=True)

# ---- score all windows ----
probs_all = model_gru.predict(X2, batch_size=512, verbose=0).reshape(-1).astype(float)

tri = meta2.copy().reset_index(drop=True)
tri["p_closed"] = probs_all
tri["y_true_window"] = np.asarray(y2, dtype=int)

# ---- helper: recent window filter (last N months of windows per business, based on end_month) ----
RECENT_MONTHS = 12  # define "recent" windows as those ending within the last 12 months of that business's observed timeline

def add_business_level_features(g: pd.DataFrame) -> pd.Series:
    g = g.sort_values("end_month")

    p_last = float(g["p_closed"].iloc[-1])
    p_max  = float(g["p_closed"].max())
    p_mean = float(g["p_closed"].mean())
    n_windows = int(len(g))

    # define recent cutoff relative to that business's last observed end_month
    end_last = pd.to_datetime(g["end_month"].iloc[-1])
    recent_cut = end_last - pd.DateOffset(months=RECENT_MONTHS - 1)
    g_recent = g[pd.to_datetime(g["end_month"]) >= recent_cut]

    if len(g_recent) == 0:
        p_recent_max = p_last
    else:
        p_recent_max = float(g_recent["p_closed"].max())

    # combo score: emphasize recent spike more than stale history
    # (this is what improved your Top-K metrics)
    p_combo = (0.65 * p_recent_max) + (0.35 * p_last)

    # business truth label in THIS dataset:
    # y_business = 1 if business has >=1 positive (y=1) window, else 0
    y_business = int((g["y_true_window"] == 1).any())

    return pd.Series({
        "status": g["status"].iloc[0],
        "end_month_last": end_last,
        "p_last": p_last,
        "p_recent_max": p_recent_max,
        "p_max": p_max,
        "p_mean": p_mean,
        "p_combo": float(p_combo),
        "n_windows": n_windows,
        "y_business": y_business
    })

biz_triage = (
    tri.groupby("business_id", as_index=False)
       .apply(add_business_level_features)
       .reset_index(drop=True)
)

# ---- choose your final risk score (best performer in your run) ----
biz_triage["risk_score"] = biz_triage["p_recent_max"]

# ---- bucketize risk for dashboard ----
# Use quantiles so buckets are stable-ish across runs
q = biz_triage["risk_score"].quantile([0.80, 0.90, 0.95]).to_dict()
q80, q90, q95 = float(q[0.80]), float(q[0.90]), float(q[0.95])

def bucket(v: float) -> str:
    if v >= q95: return "very_high"
    if v >= q90: return "high"
    if v >= q80: return "medium"
    return "low"

biz_triage["risk_bucket"] = biz_triage["risk_score"].map(bucket)

# ---- sort + save full triage ----
biz_triage = biz_triage.sort_values("risk_score", ascending=False).reset_index(drop=True)

all_path = os.path.join(OUT_DIR, "gru_business_triage_all.csv")
biz_triage.to_csv(all_path, index=False)
print("Saved:", all_path)

# ---- workload metrics + top lists (by percent) ----
n = len(biz_triage)
pos = int((biz_triage["y_business"] == 1).sum())
pos_rate = pos / max(1, n)
print("\nBusinesses:", n, " Positives:", pos, " Pos rate:", pos_rate)

print("\nTop-K% workload metrics (sorted by risk_score desc):")
for pct in [0.5, 1, 2, 5, 10, 15, 20]:
    k = max(1, int(round(n * (pct / 100.0))))
    topk = biz_triage.head(k)
    tp = int((topk["y_business"] == 1).sum())
    precision = tp / max(1, k)
    recall = tp / max(1, pos)

    thr = float(topk["risk_score"].iloc[-1])
    print(f"Top {pct:>4}% (k={k:>4})  thr>={thr:.4f}  precision={precision:.3f}  recall={recall:.3f}  tp={tp}")

# ---- save top 5% and top 10% convenience files ----
k5 = max(1, int(round(n * 0.05)))
k10 = max(1, int(round(n * 0.10)))

top5 = biz_triage.head(k5).copy()
top10 = biz_triage.head(k10).copy()

top5_path = os.path.join(OUT_DIR, "gru_business_triage_all_top5pct.csv")
top10_path = os.path.join(OUT_DIR, "gru_business_triage_all_top10pct.csv")

top5.to_csv(top5_path, index=False)
top10.to_csv(top10_path, index=False)

print("Saved:", top5_path)
print("Saved:", top10_path)

# ---- show head ----
display_cols = ["business_id","status","end_month_last","risk_score","p_recent_max","p_last","p_max","p_mean","n_windows","risk_bucket","y_business"]
print("\nTop 30 triage rows:")
print(biz_triage[display_cols].head(30))